In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import PIL
import json
from urbanComponent import *
import math
from vedo import download, Points, Axes, show

# from persim import plot_diagrams
# from ripser import ripser, lower_star_img

In [2]:
# f = open('../../../public/data/manhattan_wo_cells/buildings_s_joined.json', 'r')

# data = json.load(f)

# joinedLayerIndex = 0

# values = []

# for joinedObject in data['joinedObjects']:
#     if(joinedObject['joinedLayerIndex'] == joinedLayerIndex):
#         values = joinedObject['otherValues']
#         break

# f.close()

# uc = UrbanComponent()

# uc.setWorkDir("manhattan_wo_cells")
# uc.addLayerFromJsonFile("../../../public/data/manhattan_wo_cells/buildings_s.json")

# buildingsJson = uc.layers['json'][0]

# # f = open('./buildings.json', 'w')

# # f.write(json.dumps(buildingsJson))

# # f.close()

# latitude = []
# longitude = []
# height = []

# for geometry in buildingsJson['data']:
#     for i in range(int(len(geometry['geometry']['coordinates'])/3)):
#         latitude.append(geometry['geometry']['coordinates'][i*3])
#         longitude.append(geometry['geometry']['coordinates'][i*3+1])
#         height.append(geometry['geometry']['coordinates'][i*3+2])

# df = pd.DataFrame(data={'latitude': latitude, 'longitude': longitude, 'height': height})

# m1 = df[['latitude','longitude','height']].to_numpy()[:,1]
# m2 = df[['latitude','longitude','height']].to_numpy()[:,0]
# m3 = df[['latitude','longitude','height']].to_numpy()[:,2]

# xmin = m1.min()
# xmax = m1.max()
# ymin = m2.min()
# ymax = m2.max()
# zmin = m3.min()
# zmax = m3.max()

# values = np.array(values).reshape((len(values),))

# wmin = values.min()
# wmax = values.max()

# # X, Y, Z, W = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:50, wmin:wmax:0.1]

# X, Y, Z = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:50]

# # positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()]) # grid positions

# positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]) # grid positions

# # data_points = np.vstack([m1, m2, m3, values]) 

# data_points = np.vstack([m1, m2, m3]) 

# kernel = scipy.stats.gaussian_kde(data_points, weights=values)

# sampled_values = np.reshape(kernel(positions).T, X.shape)


In [3]:
# flat_grid_positions = []

# for i in range(len(positions[0,:])):
#     flat_grid_positions.append(positions[1,i])
#     flat_grid_positions.append(positions[0,i])
#     flat_grid_positions.append(positions[2,i])

# grid_points = {'grid': flat_grid_positions, 'values': sampled_values.flatten().tolist()}

# json_object = json.dumps(grid_points)

# file = open('grid_points.json', 'w')

# file.write(json_object)

# file.close()

### 2D plot

In [4]:
# fig, ax = plt.subplots()
# ax.imshow(np.rot90(sampled_values), cmap=plt.cm.YlOrBr, extent=[xmin, xmax, ymin, ymax])
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
# plt.show()

### 3D plot

In [5]:
# from vedo import download, Points, Axes, show
# from vedo.pyplot import CornerHistogram

# # pts = np.c_[sampled_values[0,:,:],sampled_values[:,0,:],sampled_values[:,:,0]] # form an array of 3d points from the columns

# # pts = Points(positions[0,:].tolist(), positions[1,:].tolist(), positions[2,:].tolist())
# pts = Points(positions)

# pts.pointdata["scals"] = sampled_values.flatten()

# vol = pts.tovolume(kernel='shepard', N=4, dims=(90,90,90))

# vol.c(["maroon","g","b"])        # set color   transfer function
# vol.alpha([0.3, 0.9])            # set opacity transfer function
# #vol.alpha([(0.3,0.3), (0.9,0.9)]) # alternative way, by specifying (xscalar, alpha)
# # vol.alpha_unit(0.5)              # make the whole object less transparent (default is 1)

# # replace voxels of specific range with a new value
# # vol.threshold(above=0.3, below=0.4, replace=0.9)
# # Note that scalar range now has changed (you may want to reapply vol.c().alpha())

# ch = CornerHistogram(vol, pos="bottom-left")

# # vol.add_scalarbar3d('Height is the voxel scalar', s=[None,1])
# # vol.scalarbar.rotate_x(90).pos(1.15,1,0.5)

# show(pts, vol, ch, __doc__, axes=1, elevation=-90).close()

### Creates points layer from best points

In [6]:
def break_into_binary(filepath, filename, data, types, dataTypes):

    for index, type in enumerate(types):

        readCoords = 0

        floatList = []

        for i in range(len(data['data'])):
            geometry = data['data'][i]['geometry']

            newValue = [readCoords, len(geometry[type])] # where this vector starts and its size

            readCoords += len(geometry[type])

            floatList += geometry[type].copy()

            geometry[type] = newValue

        fout = open(os.path.join(filepath,filename+'_'+type+'.data'), 'wb')

        buf = struct.pack(str(len(floatList))+dataTypes[index], *floatList)

        fout.write(buf)
        fout.close()

        json_object = json.dumps(data)

        with open(os.path.join(filepath,filename+".json"), "w") as outfile:
            outfile.write(json_object)

In [7]:
file = open('grid_points.json', 'r')

grid_points = json.load(file)

file.close()

id = 'grid_points'
type = 'POINTS_LAYER'
styleKey = 'building'
renderStyle = ['FLAT_COLOR']
selectable = False
visible = True
skip = False

layer = {
    'id': id,
    'type': type,
    'styleKey': styleKey,
    'renderStyle': renderStyle,
    'selectable': selectable,
    'visible': visible,
    'skip': skip
}

layer['data'] = [
    {
        'geometry': {
            'coordinates': grid_points['grid']
        }
    }
]

types = []
dataTypes = []

if('coordinates' in layer['data'][0]['geometry']):
    types.append("coordinates")
    dataTypes.append("d")

if('normals' in layer['data'][0]['geometry']):
    types.append("normals")
    dataTypes.append("f")

if('indices' in layer['data'][0]['geometry']):
    types.append("indices")
    dataTypes.append("I")

if('ids' in layer['data'][0]['geometry']):
    types.append("ids")
    dataTypes.append("I")

break_into_binary('./', layer['id'], layer, types, dataTypes)


### Example 1

In [8]:
# from scipy import stats
# import numpy as np
# import matplotlib.pyplot as plt

# bins = 21

# #Define a Gaussian mixture to draw samples from
# num_samples = 10000
# xmin, xmax = -10, 8
# #Weight attributed to each component of the mixture
# gaussian_weights = np.array([2, 1], dtype=np.float)
# gaussian_weights /= np.sum(gaussian_weights)
# #Mean and std of each mixture
# gaussian_means = np.array([-1, 1])
# gaussian_std = np.array([2, 1])
# #Observation probability of each mixture
# gaussian_observation = np.array([1, .5])

# #How many samples belong to each mixture?
# gaussian_samples = np.random.multinomial(num_samples, gaussian_weights)
# samples = []
# weights = []
# #Generate samples and observed samples for each mixture component
# for n, m, s, o in zip(gaussian_samples, gaussian_means, gaussian_std, gaussian_observation):
#     _samples = np.random.normal(m, s, (n, 2))
#     _samples = _samples[o > np.random.uniform(size=n)]
#     samples.extend(_samples)
#     weights.extend(np.ones(len(_samples)) / o)

# #Renormalise the sample weights
# weights = np.array(weights, np.float)
# weights /= np.sum(weights)
# samples = np.transpose(samples)

# #Evaluate the true pdf on a grid
# x = np.linspace(xmin, xmax, 100)
# xx, yy = np.meshgrid(x, x)
# true_pdf = 0
# for w, m, s in zip(gaussian_weights, gaussian_means, gaussian_std):
#     true_pdf = true_pdf + w * stats.norm(m, s).pdf(xx) * stats.norm(m, s).pdf(yy)

# #Evaluate the kde on a grid
# pdf = stats.gaussian_kde(samples, weights=weights)
# zz = pdf((np.ravel(xx), np.ravel(yy)))
# zz = np.reshape(zz, xx.shape)

# kwargs = dict(extent=(xmin, xmax, xmin, xmax), cmap='hot', origin='lower')
# #Plot the true pdf
# plt.subplot(221)
# plt.imshow(true_pdf.T, **kwargs)
# plt.title('true PDF')

# #Plot the kde
# plt.subplot(222)
# plt.imshow(zz.T, **kwargs)
# plt.title('kde')
# plt.tight_layout()

# #Plot a histogram
# ax = plt.subplot(223)
# plt.hist2d(samples[0], samples[1], bins, ((xmin, xmax), (xmin, xmax)), 
#            True, weights, cmap='hot')
# ax.set_aspect(1)
# plt.title('histogram')
# plt.tight_layout()
# plt.show()

### Example 2 (3D)

In [9]:
# def measure(n):
#     m1 = np.random.normal(size=n)
#     m2 = np.random.normal(scale=0.5, size=n)
#     m3 = np.random.normal(scale=0.8, size=n)
#     return m1+m2+m3, m1-m2-m3, m1+m2-m3

# m1, m2, m3 = measure(2000)
# xmin = m1.min()
# xmax = m1.max()
# ymin = m2.min()
# ymax = m2.max()
# zmin = m3.min()
# zmax = m3.max()

In [10]:
# X, Y, Z = np.mgrid[xmin:xmax:100j, ymin:ymax:100j, zmin:zmax:100j]
# positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])
# values = np.vstack([m1, m2, m3])
# kernel = scipy.stats.gaussian_kde(values)
# W = np.reshape(kernel(positions).T, X.shape)

### Example 2 (2D)

In [11]:
# def measure(n):
#     m1 = np.random.normal(size=n)
#     m2 = np.random.normal(scale=0.5, size=n)
#     return m1+m2, m1-m2

# m1, m2 = measure(2000)
# xmin = m1.min()
# xmax = m1.max()
# ymin = m2.min()
# ymax = m2.max()

In [12]:
# xmax-xmin

In [13]:
# X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j, zmin:zmax:100j]
# positions = np.vstack([X.ravel(), Y.ravel()])
# values = np.vstack([m1, m2])
# kernel = scipy.stats.gaussian_kde(values)
# Z = np.reshape(kernel(positions).T, X.shape)

In [14]:
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,
#           extent=[xmin, xmax, ymin, ymax])
# ax.plot(m1, m2, 'k.', markersize=2)
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
# plt.show()